# Dinamično programiranje

## Stolpi

![](../zapiski/slike/stolpi.png)

In [1]:
def stolpi(n):
    if n < 0:
        return 0
    if n == 0:
        return 1
    else:
        return stolpi(n - 1) + stolpi(n - 2) + stolpi(n - 3)

In [2]:
stolpi(10)

274

In [3]:
[stolpi(i) for i in range(10)]

[1, 1, 2, 4, 7, 13, 24, 44, 81, 149]

In [4]:
stolpi(30)

53798080

In [5]:
def stolpi_od_spodaj_navzgor(n):
    st = {-2: 0, -1: 0, 0: 1}
    for i in range(1, n + 1):
        st[i] = st[i - 1] + st[i - 2] + st[i - 3]
    return st[n]

In [6]:
[stolpi_od_spodaj_navzgor(i) for i in range(10)]

[1, 1, 2, 4, 7, 13, 24, 44, 81, 149]

In [7]:
stolpi_od_spodaj_navzgor(1000)

2758842807766486252615892411656158645133100149652696210351601845036392978912293462801016485671033253921841350537004356434253826361707295202024537559785200706502368152965047761644352316799391470273906561574500883480570560512982435681502330814068718832813973880527601

In [8]:
def stolpi_iz_sestdesetih(n):
    a, b, c = 0, 0, 1
    for _ in range(1, n + 1):
        a, b, c = b, c, a + b + c
    return c

In [9]:
[stolpi_iz_sestdesetih(i) for i in range(10)]

[1, 1, 2, 4, 7, 13, 24, 44, 81, 149]

## ↓/→ pot z najmanjšo vsoto

<table>
<tr><td><strong>131</strong></td><td> 673 </td><td> 234 </td><td> 103 </td><td> 18</td></tr>
<tr><td><strong>201</strong></td><td><strong>96</strong></td><td> <strong>342</strong></td><td> 965 </td><td> 150</td></tr>
<tr><td> 630 </td><td> 803 </td><td><strong>746</strong></td><td><strong>422</strong></td><td> 111</td></tr>
<tr><td> 537 </td><td> 699 </td><td> 497 </td><td><strong>121</strong></td><td> 956</td></tr>
<tr><td> 805 </td><td> 732 </td><td> 524 </td><td><strong>37</strong> </td><td><strong>331</strong></td></tr>
</table>

In [10]:
mat = [
    [131, 673, 234, 103, 18],
    [201, 96, 342, 965, 150],
    [630, 803, 746, 422, 111],
    [537, 699, 497, 121, 956],
    [805, 732, 524, 37, 331],
]

In [11]:
def najcenejsa_pot(mat):
    m, n = len(mat), len(mat[0])
    cena = [[None for _ in range(n)] for _ in range(m)]
    cena[-1][-1] = mat[-1][-1]
    for j in range(n - 2, -1, -1):
        cena[-1][j] = mat[-1][j] + cena[-1][j + 1]
    for i in range(m - 2, -1, -1):
        cena[i][-1] = mat[i][-1] + cena[i + 1][-1]
        for j in range(n - 2, -1, -1):
            cena[i][j] = mat[i][j] + min(cena[i][j + 1], cena[i + 1][j])
    return cena[0][0]

Še ena varianta z malo lepšimi indeksi, ki išče najcenejšo pot do vsake točke iz zgornjega levega vogala, namesto iz vsake točke do spodnjega desnega vogala.

In [12]:
def najcenejsa_pot(mat):
    m, n = len(mat), len(mat[0])
    cena = [[None for _ in range(n)] for _ in range(m)]
    cena[0][0] = mat[0][0]
    for j in range(1, n):
        cena[0][j] = mat[0][j] + cena[0][j - 1]
    for i in range(1, m):
        cena[i][0] = mat[i][0] + cena[i - 1][0]
        for j in range(1, n):
            cena[i][j] = mat[i][j] + min(cena[i][j - 1], cena[i - 1][j])
    return cena[-1][-1]

In [13]:
najcenejsa_pot(mat)

2427

In [14]:
def dodaj(x, y, par):
    xs, ys = par
    return x + xs, y + ys

def najcenejsa_pot(mat):
    m, n = len(mat), len(mat[0])
    cene_poti = [[None for _ in range(n)] for _ in range(m)]
    cene_poti[-1][-1] = (mat[-1][-1], "∙")
    for j in range(n - 2, -1, -1):
        cene_poti[-1][j] = dodaj(mat[-1][j], "→", cene_poti[-1][j + 1])
    for i in range(m - 2, -1, -1):
        cene_poti[i][-1] = dodaj(mat[i][-1], "↓", cene_poti[i + 1][-1])
        for j in range(n - 2, -1, -1):
            cene_poti[i][j] = min(
                dodaj(mat[i][j], "→", cene_poti[i][j + 1]),
                dodaj(mat[i][j], "↓", cene_poti[i + 1][j])
            )
    return cene_poti[0][0]

In [15]:
najcenejsa_pot(mat)

(2427, '↓→→↓→↓↓→∙')

## Memoizacija

In [16]:
from functools import cache

In [17]:
@cache
def cache_stolpi(n):
    if n < 0:
        return 0
    if n == 0:
        return 1
    else:
        return cache_stolpi(n - 1) + cache_stolpi(n - 2) + cache_stolpi(n - 3)

In [18]:
cache_stolpi(500)

1306186569702186634983475450062372018715120191391192207156664343051610913971927959744519676992404852130396504615663042713312314219527

In [19]:
def cache_najcenejsa_pot(mat):
    m, n = len(mat), len(mat[0])

    @cache
    def pomozna(i, j):
        if i >= m or j >= n:
            return float("inf")
        elif i == m - 1 and j == n - 1:
            return mat[i][j]
        else:
            return mat[i][j] + min(pomozna(i, j + 1), pomozna(i + 1, j))
    return pomozna(0, 0)

In [20]:
cache_najcenejsa_pot(mat)

2427

In [21]:
ze_izracunane = {}
def stolpi_s_slovarjem(n):
    if n in ze_izracunane:
        return ze_izracunane[n]
    if n < 0:
        return 0
    if n == 0:
        return 1
    else:
        st = stolpi_s_slovarjem(n - 1) + stolpi_s_slovarjem(n - 2) + stolpi_s_slovarjem(n - 3)
        ze_izracunane[n] = st
        return st    

In [22]:
stolpi_s_slovarjem(20)

121415

In [23]:
ze_izracunane[42] = 500

In [24]:
stolpi_s_slovarjem(42)

500

In [25]:
def dodaj_spomin(f):
    spomin = {}
    def f_s_spominom(x):
        if x in spomin:
            return spomin[x]
        else:
            y = f(x)
            spomin[x] = y
            return y
    return f_s_spominom

In [26]:
def odgovor(x):
    print(f"Računam odgovor({x})...")
    return 42

In [27]:
odgovor(10)

Računam odgovor(10)...


42

In [28]:
def stolpi_brez_spomina(n):
    if n < 0:
        return 0
    if n == 0:
        return 1
    else:
        return stolpi_brez_spomina(n - 1) + stolpi_brez_spomina(n - 2) + stolpi_brez_spomina(n - 3)
stolpi_s_spominom = dodaj_spomin(stolpi_brez_spomina)

In [29]:
stolpi_s_spominom(30)

53798080

In [30]:
def stolpi_v_resnici_s_spominom(n):
    if n < 0:
        return 0
    if n == 0:
        return 1
    else:
        return stolpi_v_resnici_s_spominom(n - 1) + stolpi_v_resnici_s_spominom(n - 2) + stolpi_v_resnici_s_spominom(n - 3)
stolpi_v_resnici_s_spominom = dodaj_spomin(stolpi_v_resnici_s_spominom)

In [31]:
stolpi_v_resnici_s_spominom(50)

10562230626642

In [32]:
@dodaj_spomin
def stolpi_z_dekoratorjem(n):
    if n < 0:
        return 0
    if n == 0:
        return 1
    else:
        return stolpi_z_dekoratorjem(n - 1) + stolpi_z_dekoratorjem(n - 2) + stolpi_z_dekoratorjem(n - 3)

In [33]:
stolpi_z_dekoratorjem(50)

10562230626642

In [34]:
def funkcija_visjega_reda(f):
    return f(42)

In [35]:
@funkcija_visjega_reda
def kvadriraj(x):
    return x * x

In [36]:
kvadriraj

1764

## Levenshteinova razdalja

In [37]:
def pot_od_prazne_besede(beseda):
    return [""] if beseda == "" else pot_od_prazne_besede(beseda[1:]) + [beseda]

def pot_do_prazne_besede(beseda):
    return [""] if beseda == "" else [beseda] + pot_do_prazne_besede(beseda[1:])

def dodaj_znak_vsakemu_koraku(znak, pot):
    return [znak + korak for korak in pot]

In [38]:
from functools import cache

@cache
def najkrajsa_pot(beseda1, beseda2):
    if beseda1 == "":
        return pot_od_prazne_besede(beseda2)
    elif beseda2 == "":
        return pot_do_prazne_besede(beseda1)
    elif beseda1[0] == beseda2[0]:
        # Xabc ~> ... ~> Xdef
        # kjer je abc ~> ... ~> def
        pot_med_repoma = najkrajsa_pot(beseda1[1:], beseda2[1:])
        return dodaj_znak_vsakemu_koraku(beseda1[0], pot_med_repoma)
    else:
        # Xabc ~> ... ~> Ydef

        # X zamenjamo z Y
        # Xabc ~> Yabc ~> Y... ~> Ydef
        zamenjamo = najkrajsa_pot(beseda2[0] + beseda1[1:], beseda2)

        # X pobrišemo
        # Xabc ~> abc ~> ... ~> Ydef
        pobrisemo = najkrajsa_pot(beseda1[1:], beseda2)

        # Y dodamo na začetek
        # Xabc ~> YXabc ... ~> Ydef
        dodamo = najkrajsa_pot(beseda2[0] + beseda1, beseda2)

        return [beseda1] + min(zamenjamo, pobrisemo, dodamo, key=len)

In [39]:
najkrajsa_pot("matematika", "programiranje")

['matematika',
 'patematika',
 'prtematika',
 'proematika',
 'progmatika',
 'progratika',
 'programika',
 'programira',
 'programirae',
 'programiraje',
 'programiranje']